In [3]:
import pandas as pd
from pandas.io.json import loads, json_normalize

# challenge specifies to load fullVisitorId as a string to guarantee uniqueness
id_to_str = {'fullVisitorId': str}

train = pd.read_csv('./all/train.csv', dtype=id_to_str)
test  = pd.read_csv('./all/test.csv', dtype=id_to_str)

In [4]:
def parse_json_cols(df):
    for col in ['device', 'geoNetwork', 'totals', 'trafficSource']:
        # load the json and separate
        df_json = json_normalize(df[col].apply(loads), sep='/')
        df_json.columns = [(col + '/' + c) for c in df_json.columns]
        df.drop(columns=col, inplace=True)
        df = pd.concat([df, df_json], axis=1)
    return df

#
train = parse_json_cols(train)
test  = parse_json_cols(test)

---
# Examine data

### Dates are not parsed and visitStartTime is in POSIX time
    * visitStartTime makes Date redundant (more information encoded)
    * need to get continuous time data (hour, minute)

In [5]:
from datetime import datetime

def parse_dates(df):
    df['visitStartTime'] = df['visitStartTime'].apply(lambda time: datetime.fromtimestamp(time))
    df['year']  = df['visitStartTime'].dt.year
    df['month'] = df['visitStartTime'].dt.month
    df['day']   = df['visitStartTime'].dt.day
    df['hour']  = df['visitStartTime'].dt.hour + (df['visitStartTime'].dt.minute/60)
    df.drop(columns=['date', 'visitStartTime'], inplace=True)

    return df

#
train = parse_dates(train)
test  = parse_dates(test)

### Several columns have NaN values
* Every value in totals should have a lower bound of 0.
* Will fill remainder with most frequent value

In [6]:
nan_series = train.isna().any()
nan_true   = nan_series[nan_series == True]
nan_true

totals/bounces                                  True
totals/newVisits                                True
totals/pageviews                                True
totals/transactionRevenue                       True
trafficSource/adContent                         True
trafficSource/adwordsClickInfo/adNetworkType    True
trafficSource/adwordsClickInfo/gclId            True
trafficSource/adwordsClickInfo/isVideoAd        True
trafficSource/adwordsClickInfo/page             True
trafficSource/adwordsClickInfo/slot             True
trafficSource/campaignCode                      True
trafficSource/isTrueDirect                      True
trafficSource/keyword                           True
trafficSource/referralPath                      True
dtype: bool

In [7]:
def clean_nans(df):
    totals_cols = list(filter(lambda c : c.startswith('totals'), df.columns))
    for i in totals_cols:
        df[i] = df[i].astype(float)
        df[i].fillna(0, inplace=True)
   
    nan_series = df.isna().any()
    nan_true   = nan_series[nan_series == True]
    remaining_nan_cols = list(set(nan_true.index) - set(totals_cols))
    
    for i in remaining_nan_cols:
        # fill with most frequent value
        df[i].fillna(df[i].value_counts().index[0], inplace=True)
        
    return df

#
train = clean_nans(train)
test  = clean_nans(test)

### Several columns have 1 unique value
    * Provide no information, so will drop

In [8]:
for col in train:
    if len(train[col].unique()) == 1:
        print(col)

socialEngagementType
device/browserSize
device/browserVersion
device/flashVersion
device/language
device/mobileDeviceBranding
device/mobileDeviceInfo
device/mobileDeviceMarketingName
device/mobileDeviceModel
device/mobileInputSelector
device/operatingSystemVersion
device/screenColors
device/screenResolution
geoNetwork/cityId
geoNetwork/latitude
geoNetwork/longitude
geoNetwork/networkLocation
totals/visits
trafficSource/adwordsClickInfo/criteriaParameters
trafficSource/adwordsClickInfo/isVideoAd
trafficSource/campaignCode
trafficSource/isTrueDirect


In [9]:
def drop_single_val_cols(df):
    for col in df:
        if len(df[col].unique()) == 1:
            df.drop(columns=[col], inplace=True)
    return df

#
train = drop_single_val_cols(train)
test  = drop_single_val_cols(test)

# A lot of data is categorical
* Thinks like referal links, locations, and keywords are categorical and need to be represented as such
    * Create features depending on importance of certain values
    * Dropping certain columns with seemingly redundant or unnecessary data
    * Will use one-hot encoding for the rest, bucketing less frequent values

In [10]:
# keep an unencoded copy for visualization
unencoded_train = train.copy(deep=True)

categories = list(train.select_dtypes(include='object').columns)
categories.remove('fullVisitorId')
categories

['channelGrouping',
 'sessionId',
 'device/browser',
 'device/deviceCategory',
 'device/operatingSystem',
 'geoNetwork/city',
 'geoNetwork/continent',
 'geoNetwork/country',
 'geoNetwork/metro',
 'geoNetwork/networkDomain',
 'geoNetwork/region',
 'geoNetwork/subContinent',
 'trafficSource/adContent',
 'trafficSource/adwordsClickInfo/adNetworkType',
 'trafficSource/adwordsClickInfo/gclId',
 'trafficSource/adwordsClickInfo/page',
 'trafficSource/adwordsClickInfo/slot',
 'trafficSource/campaign',
 'trafficSource/keyword',
 'trafficSource/medium',
 'trafficSource/referralPath',
 'trafficSource/source']

## Feature: Contains "Google"
* For the following columns, many rows with non-zero revenue have a string containing some variation of "Google"
    * trafficSource/keyword
    * trafficSource/adContent
    * trafficSource/adwordsClickInfo/adNetworkType
    * trafficSource/source

In [11]:
import re

google_cols = ['trafficSource/keyword', 
               'trafficSource/adContent', 
               'trafficSource/adwordsClickInfo/adNetworkType', 
               'trafficSource/source']

def contains_google(df):
    for i in google_cols:
        if i not in df.columns:
            continue
        df['{0}/contains_google'.format(i)] =\
        df[i].apply(lambda x: bool(re.search('google', x, re.IGNORECASE)))
        df.drop(columns=[i], inplace=True)
    return df

#
train = contains_google(train)
test  = contains_google(test)

---
* Removing some columns for now:

In [12]:
to_drop = ['sessionId',
           'trafficSource/adwordsClickInfo/gclId',
           'trafficSource/adwordsClickInfo/page',
           'trafficSource/campaign',
           'trafficSource/referralPath',
           'geoNetwork/networkDomain']

def drop_columns(df):
    for c in to_drop:
        if c in df.columns:
            df.drop(columns=[c], inplace=True)
    return df
        
#
train = drop_columns(train)
test  = drop_columns(test)

In [13]:
categories = list(train.select_dtypes(include='object').columns)
categories.remove('fullVisitorId')
categories

['channelGrouping',
 'device/browser',
 'device/deviceCategory',
 'device/operatingSystem',
 'geoNetwork/city',
 'geoNetwork/continent',
 'geoNetwork/country',
 'geoNetwork/metro',
 'geoNetwork/region',
 'geoNetwork/subContinent',
 'trafficSource/adwordsClickInfo/slot',
 'trafficSource/medium']

In [14]:
def one_hot(df):
    for i in categories:
        if i not in df.columns:
            continue
        value_counts = df[i].value_counts()
        if len(value_counts) > 4:
            bound = value_counts[3]
            bucket = value_counts[value_counts <= bound].index
            if len(bucket) >= 1:
                df[i] = df[i].replace(bucket, 'bucket')
    df = pd.get_dummies(df, columns=categories)
    return df

#
train = one_hot(train)
test  = one_hot(test)

---
* Target specified as ln(total transaction revenue + 1)
    * Need to convert transactionRevenue column to accomodate

In [15]:
from numpy import log1p

# to make the line more readable
i = 'totals/transactionRevenue'

train[i] = train[i].apply(lambda revenue: log1p(revenue))
unencoded_train[i] = unencoded_train[i].apply(lambda revenue: log1p(revenue))

# External Dataset: OECD Consumer Confidence Index

OECD - Total from Jan 2016 - Sep 2018

https://data.oecd.org/leadind/consumer-confidence-index-cci.htm#indicator-chart


* OECD (2018), Consumer confidence index (CCI) (indicator). doi: 10.1787/46434d78-en (Accessed on 11 October 2018)

In [16]:
def add_cci(df):
    cci = pd.read_csv('./all/oecd_total_cci.csv', index_col=0)
    cci_dict = cci.to_dict()['Value']
    df['year-month']  = df['year'].astype(str) + '-' + df['month'].astype(str)
    
    df['cci'] = df['year-month'].map(cci_dict)
    df['cci'].fillna(df['cci'].median(), inplace=True)
    
    df.drop(columns=['year-month'], inplace=True)
    return df

train = add_cci(train)
test  = add_cci(test)
unencoded_train = add_cci(unencoded_train)

#### Remove differences in columns from training set
* Only column difference should be totals/transactionRevenue

In [21]:
train_columns = train.columns
test_columns  = test.columns
in_both = list(set(train_columns) & set(test_columns))

train_columns = in_both
test_columns  = in_both[:]
train_columns.append('totals/transactionRevenue')

train = train[train_columns]
test  = test[test_columns]

#### Save Preprocessed DataFrames

In [22]:
train.to_csv('./all/p_train.csv', index=False)
test.to_csv('./all/p_test.csv', index=False)
unencoded_train.to_csv('./all/unencoded_train.csv', index=False)

---
---
---
# Load Preprocessed DataFrames

In [ ]:
import pandas as pd

id_to_str = {'fullVisitorId': str}

train = pd.read_csv('./all/p_train.csv', dtype=id_to_str)
test  = pd.read_csv('./all/p_test.csv', dtype=id_to_str)
unencoded_train = pd.read_csv('./all/unencoded_train.csv', dtype=id_to_str)

---
# Visualize

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

%matplotlib inline

In [ ]:
no_cats = unencoded_train.select_dtypes(exclude='object')

i = 'totals/transactionRevenue'
columns = list(no_cats.columns)
columns.remove(i); columns.append(i)

no_cats = no_cats[columns]

no_cats_corr = no_cats.corr()
plt.figure(figsize=(10, 8))
plt.title('Correlation Matrix (Non-Categorical Variables)', fontsize=15)
sns.heatmap(no_cats_corr, cmap='coolwarm', annot=True, vmin=-1, vmax=1, 
            cbar_kws=dict(ticks=list(np.arange(-1.0, 1.1, 0.25))), fmt='.2f')
plt.show()

In [ ]:
no_cats.corr()['totals/transactionRevenue'].sort_values(ascending=False)

---
### The majority of paying customers will view the site under 30 times.

In [ ]:
pv_tr = train[train['totals/transactionRevenue'] > 0]
p_99 = np.percentile(pv_tr['totals/pageviews'].sort_values(), 99)
pv_tr = pv_tr[pv_tr['totals/pageviews'] < p_99]

x_values = pv_tr['totals/pageviews']
y_values = pv_tr['totals/transactionRevenue']

xy = np.vstack([x_values, y_values]); 
density = gaussian_kde(xy)(xy)

plt.figure(figsize=(12, 10)); plt.title('Page Views of Paying Customers', fontsize=20)
plt.xticks(np.arange(0, x_values.max(), step=5)); plt.yticks(np.arange(0, y_values.max(), step=1))
plt.xlabel('Page Views', fontsize=15); plt.ylabel('Log Transaction Revenue', fontsize=15)
plt.grid(True)
plt.scatter(x_values, y_values, alpha=1, c=density, cmap='jet')
plt.show()

---
### Referalls bring in the most page views and the most revenue

In [ ]:
browser_rev = unencoded_train[['totals/transactionRevenue', 'channelGrouping']]
#browser_rev = browser_rev[browser_rev['totals/transactionRevenue'] > 0]
browser_rev = browser_rev.groupby('channelGrouping').mean()
browser_rev = browser_rev.sort_values(by='totals/transactionRevenue', ascending=False)
x = browser_rev.index
y = browser_rev['totals/transactionRevenue']
plt.figure(figsize=(10, 5))
sns.barplot(x, y).set_title('A')
plt.show()

In [ ]:
mobile_revenue = train[['totals/transactionRevenue', 'device/isMobile']]
mobile_revenue = mobile_revenue.groupby('device/isMobile').sum()
x = mobile_revenue.index
y = mobile_revenue['totals/transactionRevenue']
sns.barplot(x, y).set_title('A')
plt.show()

In [ ]:
unencoded_train['month'].value_counts()

In [ ]:
unencoded_train['channelGrouping'].value_counts()



https://support.google.com/analytics/answer/3297892?hl=en&ref_topic=6010089

In [ ]:
channels = unencoded_train[['channelGrouping', 'totals/transactionRevenue', 'month']]

plt.figure(figsize=(15,9))
plt.xticks(np.arange(0, 12+1, step=1))
channels = channels.sort_values(by='month')

sns.set(style="ticks", rc={"lines.linewidth": 6})
sns.lineplot('month', 'totals/transactionRevenue', data=channels, err_style=None, estimator='mean', palette="colorblind",  hue='channelGrouping')
plt.title('Mean Revenue By Month', fontsize=25)
plt.xlabel('Month', fontsize=20); 
plt.ylabel('Log Transaction Revenue', fontsize=20)
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=1, frameon=False)
plt.show()

In [ ]:
channels = unencoded_train[['channelGrouping', 'totals/pageviews', 'month']]
channels = channels[channels['channelGrouping'] != '(Other)']

plt.figure(figsize=(15,9))
plt.xticks(np.arange(0, 12+1, step=1))
channels = channels.sort_values(by='month')

sns.set(style="ticks", rc={"lines.linewidth": 6})
sns.lineplot('month', 'totals/pageviews', data=channels, err_style=None,  hue='channelGrouping')
plt.title('Page Views By Month and Channel Grouping', fontsize=20)
plt.xlabel('Month', fontsize=15); 
plt.ylabel('Page Views', fontsize=15)
plt.show()

---
# Train models

In [24]:
def get_tt_split(df):
    X = df.drop(columns=['totals/transactionRevenue', 'fullVisitorId'])
    y = df['totals/transactionRevenue']
    
    return train_test_split(X, y, test_size = 0.1, random_state = 3)

### Linear Regression

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
from numpy import clip

def train_lr(df):
    X_train, X_test, y_train, y_test = get_tt_split(df)
    
    model = LinearRegression()
    print('Training model: {0}...\n'.format('lr'))
    model.fit(X_train, y_train)
    test_predictions = model.predict(X_test)
    test_predictions = clip(test_predictions, a_min=0, a_max=None)
    
    mse = mean_squared_error(y_test, test_predictions)
    rmse = sqrt(mse)
    
    print('=====')
    print('RMSE:', rmse)
    print('=====\n\n==========')
    print('INTERCEPT:', model.intercept_)
    print('==========\n\n==========================')
    print('COEFFICIENTS (decreasing):')
    print('==========================')
    coefficients = list(zip(X_test.columns, model.coef_))
    coefficients.sort(key = lambda c: c[1])
    for i in coefficients[::-1]:
        print('{0}:\n\t{1}\n'.format(i[0], i[1]))
    
    
    return model

In [26]:
lr_model = train_lr(train)

Training model: lr...

=====
RMSE: 1.809177127724782
=====

INTERCEPT: -5777.532793273914

COEFFICIENTS (decreasing):
year:
	2.9235173845031905

totals/bounces:
	0.35194058116154275

totals/pageviews:
	0.26287769645484527

month:
	0.24931753904567697

geoNetwork/country_United States:
	0.2070333798108353

trafficSource/medium_referral:
	0.19246745898774345

geoNetwork/metro_bucket:
	0.12995820865938498

trafficSource/adwordsClickInfo/adNetworkType/contains_google:
	0.1194052442394892

device/operatingSystem_Macintosh:
	0.09898160981322648

cci:
	0.09117718132140583

device/deviceCategory_desktop:
	0.06827551766679271

channelGrouping_bucket:
	0.06517463559935287

geoNetwork/city_bucket:
	0.05476007742616696

trafficSource/source/contains_google:
	0.05076248709913875

trafficSource/adwordsClickInfo/slot_RHS:
	0.03689108725109238

geoNetwork/continent_bucket:
	0.032897416992357535

device/browser_bucket:
	0.029479240153955814

trafficSource/keyword/contains_google:
	0.02656662979871958



In [31]:
import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt
from numpy import clip

def train_xgb(df):
    X_train, X_test, y_train, y_test = get_tt_split(df)
    
    print('Training model: {0}...'.format('xgb'))
    model = xgb.train({}, DMatrix(X_train, y_train))
    test_predictions = model.predict(DMatrix(X_test))
    test_predictions = clip(test_predictions, a_min=0, a_max=None)
    
    mse = mean_squared_error(y_test, test_predictions)
    rmse = sqrt(mse)
    
    print('=====')
    print('RMSE:', rmse)
    return model

In [28]:
xgb_model = train_xgb(train)

Training model: xgb...
=====
RMSE: 1.6536570893039457


---
# Specify model

In [32]:
selected_model = 'xgb'

# Create Submission

In [33]:
X_test = test.drop(columns=['fullVisitorId'])

if selected_model == 'lr':
    model = train_lr(train)
elif selected_model == 'xgb':
    model = train_xgb(train)
    X_test = DMatrix(X_test)

real_predictions = model.predict(X_test)
real_predictions = clip(real_predictions, a_min=0, a_max=None)

real_df = test['fullVisitorId'].to_frame()
real_df['PredictedLogRevenue'] = real_predictions
real_df['fullVisitorId'] = real_df['fullVisitorId'].astype('str')
real_df = real_df.groupby('fullVisitorId').sum()
real_df.to_csv('submission.csv')

Training model: xgb...
=====
RMSE: 1.6516302891477297
